In [1]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage
import io
import hashlib
import time

In [2]:
from src.customer_request import CustomerRequest
from src.hasher import Hasher
from src.customer import Customer

In [3]:
# create the customer org
customer = Customer("MOD")
vars(customer)

{'org_name': 'MOD', 'password': None}

In [4]:
# the client where we store the data (can be BQ)
client = storage.Client()
bucket_name = "rdmf_mock_data"

In [5]:
# customer requests some data, they want AI and BI joined on UPRN:
cus_req = CustomerRequest(customer.org_name, "AI", "BI", "UPRN")
vars(cus_req)

{'org_name': 'MOD',
 'dataset1': 'AI',
 'dataset2': 'BI',
 'merge_col': 'UPRN',
 'req_id': 0,
 'epoch': 1709808199.9601805,
 'password_prefix': '7779680',
 'password_suffix': '684'}

In [6]:
# we set the data they request:
cus_req.set_data(client, bucket_name)

In [10]:
# we return their password to them for storage:
customer_password = cus_req.create_user_password()
print(f"customer password: {customer_password}")
print(f"note the password prefix (which we use for an identifier): {cus_req.password_prefix}")
print(f"but not the password suffix (which we store for security): {cus_req.password_suffix}")
customer.set_password(customer_password)

customer password: 7779680683666
note the password prefix (which we use for an identifier): 7779680
but not the password suffix (which we store for security): 684


In [11]:
# we encrypt the data with the combination of our keys
encrypted_data = cus_req.get_data(Hasher, customer.get_password(), ["AI_ID", "BI_ID"])
customer.set_data(encrypted_data)

In [12]:
# the customer now has data
vars(customer)

{'org_name': 'MOD',
 'password': '7779680683666',
 'data':                                                  AI_ID          UPRN  \
 0    588a3bbd5b535cc8f62e165d800ae4348b5b5c48733932...  950698168954   
 1    c74838e9af47ac9a82ff45ac6e429997d3fe8754999ba5...  950320937277   
 2    d1e13346b214ab1f4aed7ae1bc8037c56927ac4d824266...  950186421264   
 3    b4ea74251e57aa03187eb7ccbfb6595c4efabb3adb8c3f...  950680091532   
 4    f1aa8b674478774e88dfdc6a1e60e9966635ff81497164...  950396831655   
 ..                                                 ...           ...   
 995  ac1e2ca2fa315ce795cf456736483589b29687129e0266...  950887629315   
 996  c368dd369d8c3ccf333c32b9b618a0dfc0bb683f64bdb1...  950515544317   
 997  b94e4bee4041e42a4ad5651c81c31798b27b3ebfcb08af...  950244942559   
 998  c54dc8718b8dada0ea8cd33d8857afa766124c2008a8a0...  950966683600   
 999  1a1663057aa470be4e5c81522aecf60631923f67e8bb1e...  950187498346   
 
                                                  BI_ID  \
 0   